In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import re
from simpletransformers.classification import ClassificationModel
import sklearn
import itertools
import emoji

In [4]:
df = pd.read_csv(r"C:\Users\sunje\PycharmProjects\RDBTW\venv\DeepLearningTwitterBots\Transformers\train.csv")

In [5]:
columns_drop = ['keyword','location']

In [6]:
df.drop(columns=columns_drop,inplace=True)

In [7]:
df.head()

,id,text,target
0,1,Our Deeds are the Reason of this #earthquake M...,1
1,4,Forest fire near La Ronge Sask. Canada,1
2,5,All residents asked to 'shelter in place' are ...,1
3,6,"13,000 people receive #wildfires evacuation or...",1
4,7,Just got sent this photo from Ruby #Alaska as ...,1


In [8]:
fake_tweets = df[df.target == 0]
fake_tweets.shape

(4342, 3)

In [9]:
fake_tweets.head(300)

,id,text,target
15,23,What's up man?,0
16,24,I love fruits,0
17,25,Summer is lovely,0
18,26,My car is so fast,0
19,28,What a goooooooaaaaaal!!!!!!,0
...,...,...,...
503,728,@eunice_njoki aiii she needs to chill and answ...,0
507,732,I attacked Robot-lvl 19 and I've earned a tota...,0
509,735,@christinalavv @lindsay_wynn3 I just saw these...,0
515,744,@MageAvexis &lt; things. And what if we get at...,0


### Defining contractions to clean the data

In [10]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are",
"thx"   : "thanks"
}

In [11]:
def remove_contractions(text):
    return contractions[text.lower()] if text.lower() in contractions.keys() else text

In [12]:
df['text']=df['text'].apply(remove_contractions)
df.head()


,id,text,target
0,1,Our Deeds are the Reason of this #earthquake M...,1
1,4,Forest fire near La Ronge Sask. Canada,1
2,5,All residents asked to 'shelter in place' are ...,1
3,6,"13,000 people receive #wildfires evacuation or...",1
4,7,Just got sent this photo from Ruby #Alaska as ...,1


### Clean the dataset

In [13]:
def clean_dataset(text):
    # Remove hashtag while keeping hashtag text
    text = re.sub(r'#','', text)
    # Remove HTML special entities (e.g. &amp;)
    text = re.sub(r'\&\w*;', '', text)
    # Remove tickers
    text = re.sub(r'\$\w*', '', text)
    # Remove hyperlinks
    text = re.sub(r'https?:\/\/.*\/\w*', '', text)
    # Remove whitespace (including new line characters)
    text = re.sub(r'\s\s+','', text)
    text = re.sub(r'[ ]{2, }',' ',text)
    # Remove URL, RT, mention(@)
    text=  re.sub(r'http(\S)+', '',text)
    text=  re.sub(r'http ...', '',text)
    text=  re.sub(r'(RT|rt)[ ]*@[ ]*[\S]+','',text)
    text=  re.sub(r'RT[ ]?@','',text)
    text = re.sub(r'@[\S]+','',text)
    # Remove words with 4 or fewer letters
    text = re.sub(r'\b\w{1,4}\b', '', text)
    #&, < and >
    text = re.sub(r'&amp;?', 'and',text)
    text = re.sub(r'&lt;','<',text)
    text = re.sub(r'&gt;','>',text)
    # Remove characters beyond Basic Multilingual Plane (BMP) of Unicode:
    text= ''.join(c for c in text if c <= '\uFFFF') 
    text = text.strip()
    # Remove misspelling words
    text = ''.join(''.join(s)[:2] for _, s in itertools.groupby(text))
    # Remove emoji
    text = emoji.demojize(text)
    text = text.replace(":"," ")
    text = ' '.join(text.split()) 
    text = re.sub("([^\x00-\x7F])+"," ",text)
    # Remove Mojibake (also extra spaces)
    text = ' '.join(re.sub("[^\u4e00-\u9fa5\u0030-\u0039\u0041-\u005a\u0061-\u007a]", " ", text).split())
    return text

In [14]:
df['text'] =df['text'].apply(clean_dataset)
df.head()

,id,text,target
0,1,Deeds Reason earthquake ALLAH Forgive,1
1,4,Forest Ronge Canada,1
2,5,residents asked shelter place being notified o...,1
3,6,people receive wildfires evacuation orders Cal...,1
4,7,photo Alaska smoke wildfires pours school,1


In [15]:
df.shape

(7613, 3)

### Split the training and validation set

In [16]:
X_train_clean, X_test_clean, y_train_clean, y_test_clean = train_test_split(df['text'], df['target'], test_size=0.20, random_state=42)

In [17]:
train_df_clean = pd.concat([X_train_clean, y_train_clean], axis=1)
print("Shape of training data set: ", train_df_clean.shape)
print("View of data set: ", train_df_clean.head())

Shape of training data set:  (6090, 2)
View of data set:                                                     text  target
4996  Courageous honest analysis Atomic Hiroshima70 ...       1
3263          shame became engulfed flames boycottBears       0
4907  rescind medals honor given soldiers Massacre W...       1
2855  Worried about drought might affect Extreme Wea...       1
4716                           BlastPower PantherAttack       0


In [18]:
eval_df_clean = pd.concat([X_test_clean, y_test_clean], axis=1)
print("Shape of Eval data set: ", eval_df_clean.shape)

Shape of Eval data set:  (1523, 2)


### BERT Model Training

#### Set up the train arguments

In [21]:
train_args = {
    'evaluate_during_training': True,
    'logging_steps': 100,
    'num_train_epochs': 2,
    'evaluate_during_training_steps': 100,
    'save_eval_checkpoints': False,
    'train_batch_size': 32,
    'eval_batch_size': 64,
    'overwrite_output_dir': True,
    'fp16': False,
    'wandb_project': "visualization-demo"
}

In [22]:
model_BERT = ClassificationModel('bert', 'bert-base-cased', num_labels=2, use_cuda=False, cuda_device=0, args=train_args)

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

#### Train the model

In [23]:
model_BERT.train_model(train_df_clean, eval_df=eval_df_clean)

C:\Users\sunje\PycharmProjects\RDBTW\venv\lib\site-packages\simpletransformers\classification\classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/6090 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ········


...

C:\Users\sunje\PycharmProjects\RDBTW\venv\lib\site-packages\simpletransformers\classification\classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/1523 [00:00<?, ?it/s]

C:\Users\sunje\PycharmProjects\RDBTW\venv\lib\site-packages\simpletransformers\classification\classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/1523 [00:00<?, ?it/s]

(382,
 defaultdict(list,
             {'global_step': [100, 191, 200, 300, 382],
              'train_loss': [0.5244818329811096,
               0.7227049469947815,
               0.4318181574344635,
               0.3227008283138275,
               0.522789716720581],
              'mcc': [0.5988206427483633,
               0.5724149200905551,
               0.6009461498118813,
               0.584786845528686,
               0.6011243328763812],
              'tp': [451, 505, 443, 464, 465],
              'tn': [775, 697, 784, 752, 763],
              'fp': [99, 177, 90, 122, 111],
              'fn': [198, 144, 206, 185, 184],
              'auroc': [0.8486476289873879,
               0.8619165905653124,
               0.859557742416603,
               0.8561940038009541,
               0.8605846699551853],
              'auprc': [0.8491839448547343,
               0.8583518746465557,
               0.8578900174069248,
               0.8502131418651646,
               0.855800647564

#### check model performance on validation data

In [24]:
result, model_outputs, wrong_predictions = model_BERT.eval_model(eval_df_clean, acc=sklearn.metrics.accuracy_score)

C:\Users\sunje\PycharmProjects\RDBTW\venv\lib\site-packages\simpletransformers\classification\classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/1523 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/24 [00:00<?, ?it/s]

Training loss,█▅▁
auprc,▁██▂▆
auroc,▁█▇▅▇
eval_loss,▁▇▁██
fn,▇▁█▆▆
fp,▂█▁▄▃
global_step,▁▁▃▃▃▆▆█
lr,█▅▁
mcc,▇▁█▄█
tn,▇▁█▅▆
tp,▂█▁▃▃


wandb: WARNING wandb.plots.* functions are deprecated and will be removed in a future release. Please use wandb.plot.* instead.
